<h3>Turnstile Usage for all Fridays (between midnight and 4:00 am) in July 2013</h3>

The data set consits of entry and exit counters measured every four hours at every turnstile available in stations in the NYC Subway System. <p>

The format of the data for the 2013 measurements is described here: http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description_pre-10-18-2014.txt <p>


This document computes the average number of entries per station between midnight and 4am for all Fridays in July 2013. Using this information, we determine the station that had the highest average number of entries. <p>

The following link provides a list of the web files containing the data: http://web.mta.info/developers/turnstile.html

<h4>Software Description</h4>

The <b>filtered_data_frame()</b> function takes a URL pointing to raw data and generates a pandas dataframe with entry and exit numbers for all turnstile machines for a particular day and time. This function has the following arguments:<p>
Args:<p>
<b>target_url</b>: (str) URL pointing to one of the raw data files<p>
<b>day</b>: (str) The target day for which we investigate turnstile data. Format: MM-DD-YY <p>
<b>time</b>: (str) The target time for which we investigate turnstile data. Format: HH-00-00 <p>
<b>num_lines</b>: (int) The number of lines that the function scans in the raw-data file. A value of -1 indicates all lines.<p>
Returns:<p>
A pandas dataframe whose columns are identified by the following strings:<p>
"ca", "unit", "scp", "month", "day", "year", "hour", "ent", "exit"<p>
Columns "ca", "unit" and "scp" hold string values identifying respectively the station, the unit within the station, and the turnstile machine. Columns "month", "day", "year" and "hour" hold integers identifying the target day/time for turnstile data. Finally columns "ent" and "exit" hold integers that correspond to the count of entries and count of exits at the specifed day/time.

In [1]:
import urllib2
import pandas as pd

constants = {"url_for_0705": "http://web.mta.info/developers/data/nyct/turnstile/turnstile_130706.txt",
             "url_for_0712": "http://web.mta.info/developers/data/nyct/turnstile/turnstile_130713.txt",
             "url_for_0719": "http://web.mta.info/developers/data/nyct/turnstile/turnstile_130720.txt",
             "url_for_0726": "http://web.mta.info/developers/data/nyct/turnstile/turnstile_130727.txt"}


def filtered_dataframe(target_url, day, time, num_lines):
    webdata = urllib2.urlopen(target_url)

    filtered_data = []
    line_counter = 0
    for line in webdata:
        values = line.strip().split(",")

        temp_array = convert_to_array2(values, day, time)

        filtered_data.extend(temp_array)

        if num_lines != -1 and line_counter >= num_lines:
            break
        else:
            line_counter += 1

    print "[Info] Total number of scanned lines: ", line_counter

    return pd.DataFrame(filtered_data, columns=["ca", "unit", "scp", "month", "day", "year", "hour", "ent", "exit" ])

The <b>convert_to_array2()</b> function is a helper function invoked from the <b>filtered_dataframe()</b> function. The <b>convert_to_array2()</b> function actually searches for turnstile information for the target day and time in the raw-data file and returns the data as an array of lists. Each list in the array contains a single measurement for a turnstile machine. 

In [2]:
def convert_to_array2(values, day, time):
    reduced = []
    sta = values[0]
    unit = values[1]
    scp = values[2]

    for i in range(3, len(values), 5):
        vec = values[i:i+5]

        if vec[0] == day and vec[1] == time and vec[2] == 'REGULAR':
            monval, dayval, yearval = [int(val) for val in day.split("-")]
            timeval = int(time.split(":")[0])
            filtered_data = [sta, unit, scp, monval, dayval, yearval, timeval, int(vec[3]), int(vec[4])]
            reduced.append(filtered_data)

    return reduced

The <b>merge_init_final()</b> takes as input two pandas dataframes. The first dataframe contains the entry and exit counters for all the turnstile machines at a given initial day and time. The second dataframe contains the same information but for a final day and time. <p>

The function merges the two dataframes using "ca", "unit" and "scp" as the reference for this join. In other words, we end up with a table that, for every turnstile machine it contains entry/exit counts for two day-time occurrences (initial and final). <p>

The function computes the difference between entry counts, which indicates the actual number of entries for the period defined by the initial and final day-times. Similarly, it computes the difference between exit counts. It also computes the "busy-ness score", which is the sum of entries and exits during the time interval. <p>

Finally, the function checks if the difference in entry counts is negative. A negative value indicates that the entry counter has been reset during the interval. The function excludes the data from those machines whose entry counters have been reset. After performing this operation, the software performs a similar check for the difference in exit counts. Again, if the different is negative, the software removes the data from those machines. The program actually displays the list of machines whose data has been removed due to entry counter reset or exit counter reset.

In [3]:
def merge_init_final(init_df, final_df):
    # Merging the initial and final data sources using 'ca', 'unit', and 'scp' as the key..."
    mdf = pd.merge(init_df, final_df, on=["ca", "unit", "scp"])

    # Compute the difference in entry counters and exit counters. Also compute busy score
    mdf["ent_diff"] = mdf["ent_y"] - mdf["ent_x"]
    mdf["exit_diff"] = mdf["exit_y"] - mdf["exit_x"]
    mdf["busy"] = mdf["ent_diff"] + mdf["exit_diff"]

    # Process machines whose entry counts during the interval show negative values
    reset1 = mdf[mdf["ent_diff"] < 0]
    if len(reset1) > 0:
        print "[Note] Machines that had a reset of the entry counter. Data for these machines will be removed:"
        print reset1[["ca", "unit", "scp", "ent_x", "ent_y"]]
        mdf2 = mdf[mdf["ent_diff"] > 0]
    else:
        mdf2 = mdf

    # Process machines whose exit counts during the interval show negative values
    reset2 = mdf2[mdf2["exit_diff"] < 0]
    if len(reset2) > 0:
        print "[Note] Remaining machines showing a reset of the exit counter. Data for these machines will be removed:"
        print reset2[["ca", "unit", "scp", "exit_x", "exit_y"]]
        mdf3 = mdf2[mdf2["exit_diff"] > 0]
    else:
        mdf3 = mdf2

    return mdf3

The month of July 2013 has four Fridays: July 5, July 12, July 19, and July 26. For each of these Fridays, the following code uses the <b>filtered_dataframe()</b> function to compute the entry and exit counts at midnight (time set to 00:00:00). Next, it does the same for a time equal to 4:00 am. <p>

Using the <b>merge_init_final()</b> function, the code merges the previous information into a single table. The code then computes the difference in entry counters (which gives the actual number of entries in the interval). Also, it removes data records for turnstile machines whose counters have been reset. <p>

Finally, the code creates for each Friday a reduced view (called red5, red12, red19, and red26) which containes turnstile information and the actual number of entries during the time interval. 

In [5]:
 nlines = -1

# Retrieve entry and exits for July 5, 2013
res5_1 = filtered_dataframe(constants["url_for_0705"], day="07-05-13", time="00:00:00", num_lines=nlines)

res5_2 = filtered_dataframe(constants["url_for_0705"], day="07-05-13", time="04:00:00", num_lines=nlines)

# Compute the dataframe with entries, exits, differences, and busy scores
res5 = merge_init_final(res5_1, res5_2)
red5 = res5[["ca", "unit", "scp", "ent_diff"]]
red5 = red5.rename(columns={"ent_diff": "ent_diff_5"})

##############################################

# Retrieve entry and exits for July 12, 2013
res12_1 = filtered_dataframe(constants["url_for_0712"], day="07-12-13", time="00:00:00", num_lines=nlines)

res12_2 = filtered_dataframe(constants["url_for_0712"], day="07-12-13", time="04:00:00", num_lines=nlines)

# Compute the dataframe with entries, exits, differences, and busy scores
res12 = merge_init_final(res12_1, res12_2)
red12 = res12[["ca", "unit", "scp", "ent_diff"]]
red12 = red12.rename(columns={"ent_diff": "ent_diff_12"})

##############################################

# Retrieve entry and exits for July 19, 2013
res19_1 = filtered_dataframe(constants["url_for_0719"], day="07-19-13", time="00:00:00", num_lines=nlines)

res19_2 = filtered_dataframe(constants["url_for_0719"], day="07-19-13", time="04:00:00", num_lines=nlines)

# Compute the dataframe with entries, exits, differences, and busy scores
res19 = merge_init_final(res19_1, res19_2)
red19 = res19[["ca", "unit", "scp", "ent_diff"]]
red19 = red19.rename(columns={"ent_diff": "ent_diff_19"})

##############################################

# Retrieve entry and exits for July 26, 2013
res26_1 = filtered_dataframe(constants["url_for_0726"], day="07-26-13", time="00:00:00", num_lines=nlines)

res26_2 = filtered_dataframe(constants["url_for_0726"], day="07-26-13", time="04:00:00", num_lines=nlines)

# Compute the dataframe with entries, exits, differences, and busy scores
res26 = merge_init_final(res26_1, res26_2)
red26 = res26[["ca", "unit", "scp", "ent_diff"]]
red26 = red26.rename(columns={"ent_diff": "ent_diff_26"})

[Info] Total number of scanned lines:  30964
[Info] Total number of scanned lines:  30964
[Info] Total number of scanned lines:  29504
[Info] Total number of scanned lines:  29504
[Info] Total number of scanned lines:  29287
[Info] Total number of scanned lines:  29287
[Info] Total number of scanned lines:  29374
[Info] Total number of scanned lines:  29374


Finally, the next code segment computes the average number of entries for the four Fridays and sums the average values per station ("ca"). The result is a pandas Series showing the stations and their average entries for the interval between midnight and 4:00 am. <p>

Using this Series, we can now determine the station with the highest average of entries.

In [6]:
# Creating a single dataframe merging the reduced views

dfset = [red5, red12, red19, red26]

# Applying the dataframe merge function in a reduce structure
uniqdf = reduce(lambda left_df, right_df: pd.merge(left_df, right_df, on=["ca", "unit", "scp"]), dfset)

# print uniqdf

uniqdf["ave"] = uniqdf.mean(axis=1)

grouped = uniqdf.groupby("ca")["ave"].sum()
# print grouped

print "\n[Result] Station with the highest average number of entries: ", grouped.idxmax()
print "[Result] The average number of entries for this station is ", grouped[grouped.idxmax()]



[Result] Station with the highest average number of entries:  R151
[Result] The average number of entries for this station is  3130.5
